# 3차 작업

## 전국주차장 데이터셋 전처리

### 위/경도 결측값 추가

In [14]:
from concurrent.futures import ThreadPoolExecutor, as_completed 
from tqdm import tqdm
import numpy as np
import pandas as pd
import requests

df = pd.read_csv("전국주차장정보표준데이터.csv", encoding="cp949") 
# 위도 또는 경도가 결측인 행만 따로 추출
df_missing_coords = df[df['위도'].isna() | df['경도'].isna()] # isna()를 활용해 결측 -> True인 행만 df_missing_coords에 저장
df_missing_coords.head()

C:\Users\choi heung ki\AppData\Local\Temp\ipykernel_7228\2707440034.py:7: DtypeWarning: Columns (20,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("전국주차장정보표준데이터.csv", encoding="cp949")


,주차장관리번호,주차장명,주차장구분,주차장유형,소재지도로명주소,소재지지번주소,주차구획수,급지구분,부제시행구분,운영요일,...,결제방법,특기사항,관리기관명,전화번호,위도,경도,장애인전용주차구역보유여부,데이터기준일자,제공기관코드,제공기관명
0,341-2-000056,법성매립지 제5주차장,공영,노외,NaN,전라남도 영광군 법성면 법성리 1226-4,20,기타,미시행,평일+토요일+공휴일,...,NaN,NaN,전라남도 영광군청,NaN,NaN,NaN,N,2024-11-13,4970000,전라남도 영광군
1,341-2-000057,법성매립지 제6주차장,공영,노외,NaN,전라남도 영광군 법성면 법성리 1217-6,40,기타,미시행,평일+토요일+공휴일,...,NaN,NaN,전라남도 영광군청,NaN,NaN,NaN,Y,2024-11-13,4970000,전라남도 영광군
2,341-2-000058,법성매립지 제7주차장,공영,노외,NaN,전라남도 영광군 법성면 법성리 1210-8,42,기타,미시행,평일+토요일+공휴일,...,NaN,NaN,전라남도 영광군청,NaN,NaN,NaN,Y,2024-11-13,4970000,전라남도 영광군
3,341-2-000059,법성매립지 제8주차장,공영,노외,NaN,전라남도 영광군 법성면 법성리 1227-4,38,기타,미시행,평일+토요일+공휴일,...,NaN,NaN,전라남도 영광군청,NaN,NaN,NaN,N,2024-11-13,4970000,전라남도 영광군
4,341-2-000060,법성매립지 제9주차장,공영,노외,NaN,전라남도 영광군 법성면 법성리 691,10,기타,미시행,평일+토요일+공휴일,...,NaN,NaN,전라남도 영광군청,NaN,NaN,NaN,Y,2024-11-13,4970000,전라남도 영광군


### Kakao Map API를 사용한 위/경도 값 가져오기 함수

In [16]:
# 너의 Kakao REST API 키를 여기에 입력해줘
KAKAO_API_KEY = 'YOUR_API_KEY'

# 주소를 위경도로 변환하는 함수
def get_lat_lon(address):
    try:
        url = 'https://dapi.kakao.com/v2/local/search/address.json' # 카카오API 로컬의 주소검색 엔드포인트. 주소를 기반으로 좌표, 주소정보등을 반환
        headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
        params = {"query": address}
        response = requests.get(url, headers=headers, params=params)
        
        if response.status_code == 200:
            result = response.json()
            if result['documents']:
                x = float(result['documents'][0]['x'])  # 경도
                y = float(result['documents'][0]['y'])  # 위도
                return y, x
        return None, None
    except Exception as e:
        return None, None

In [18]:
# 주소 결정 (도로명 > 지번 순)
df_missing_coords["주소"] = df_missing_coords.apply(
    lambda row: row["소재지도로명주소"] if pd.notna(row["소재지도로명주소"]) else row["소재지지번주소"], # pd.notna()로 결측값이 아닌 도로명주소값 확인.
    axis=1
)

# 주소 리스트
addresses = df_missing_coords["주소"].tolist()

# 결과 저장 리스트
lat_list, lon_list = [], []

# 병렬 처리
with ThreadPoolExecutor(max_workers=20) as executor:
    futures = {executor.submit(get_lat_lon, addr): addr for addr in addresses}

    for future in tqdm(as_completed(futures), total=len(futures)):
        lat, lon = future.result()
        lat_list.append(lat)
        lon_list.append(lon)

# 위도/경도 결과 컬럼으로 추가
df_missing_coords = df_missing_coords.copy()  # 경고 방지
df_missing_coords["위도"] = lat_list
df_missing_coords["경도"] = lon_list

C:\Users\choi heung ki\AppData\Local\Temp\ipykernel_7228\3790725002.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_missing_coords["주소"] = df_missing_coords.apply(
100%|█████████████████████████████████████████████████████████████████████████████| 3862/3862 [00:14<00:00, 275.29it/s]


In [32]:
# 위경도 있는 데이터와 보완된 데이터 합치기
df_clean = df[~(df['위도'].isna() | df['경도'].isna())]
# 행 기준 결합
df_final = pd.concat([df_clean, df_missing_coords], ignore_index=True) # 예) 위/경도 있는 행 50, 위/경도 없는 행 50. concat으로 위, 아래로 합치기
df_final.to_csv("보완_전국주차장데이터_new.csv", index=False, encoding="utf-8-sig")
print("✔️ 저장 완료: 보완_주차장데이터.csv")

✔️ 저장 완료: 보완_주차장데이터.csv


### 위/경도 결측값, 중복값 제거

In [39]:
df = pd.read_csv("보완_주차장데이터_new.csv", encoding="utf-8-sig")
df_cleaned = df.dropna(subset=["위도", "경도"]) # 결측값 제거
df_cleaned = df_cleaned.drop_duplicates().reset_index(drop=True) # 중복값까지 제거 후, 인덱스 정렬해야 맞다.
df_cleaned.to_csv("보완_전국주차장데이터_결측값_중복값제거.csv", index=False, encoding='utf-8-sig')
print("✔️ 결측값, 중복값 제거 완료. 저장된 파일: 보완_주차장데이터_결측제외.csv")

C:\Users\choi heung ki\AppData\Local\Temp\ipykernel_7228\1269037816.py:1: DtypeWarning: Columns (7,20,25,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("보완_주차장데이터_new.csv", encoding="utf-8-sig")


✔️ 결측값, 중복값 제거 완료. 저장된 파일: 보완_주차장데이터_결측제외.csv


### 지역 전처리(서울/경기/인천)

In [43]:
df_cleaned = pd.read_csv("보완_전국주차장데이터_결측값_중복값제거.csv", encoding="utf-8-sig")
# 도로명주소가 있으면 사용, 없으면 지번주소 사용
df_cleaned['전체주소'] = df_cleaned.apply(
    lambda row: row['소재지도로명주소'] if pd.notna(row['소재지도로명주소']) else row['소재지지번주소'],
    axis=1
)

,주차장관리번호,주차장명,주차장구분,주차장유형,소재지도로명주소,소재지지번주소,주차구획수,급지구분,부제시행구분,운영요일,...,관리기관명,전화번호,위도,경도,장애인전용주차구역보유여부,데이터기준일자,제공기관코드,제공기관명,주소,전체주소
0,122-2-000023,대청역,공영,노외,서울특별시 강남구 개포로 623-1,NaN,162,4,미시행,평일+토요일+공휴일,...,강남구도시관리공단,1544-3113,37.494949,127.079318,NaN,2024-07-05,3220000,서울특별시 강남구,NaN,서울특별시 강남구 개포로 623-1
1,122-2-000027,일원1동(기계식),공영,노외,서울특별시 강남구 양재대로 27길 5,NaN,66,4,미시행,평일+토요일+공휴일,...,강남구도시관리공단,1544-3113,37.489565,127.081744,NaN,2024-07-05,3220000,서울특별시 강남구,NaN,서울특별시 강남구 양재대로 27길 5
2,122-2-000024,영희초교,공영,노외,서울특별시 강남구 일원로21,NaN,185,4,미시행,평일+토요일+공휴일,...,강남구도시관리공단,1544-3113,37.492300,127.081133,NaN,2024-07-05,3220000,서울특별시 강남구,NaN,서울특별시 강남구 일원로21
3,122-2-000028,대왕초교,공영,노외,서울특별시 강남구 헌릉로 618길 8,NaN,101,4,미시행,평일+토요일+공휴일,...,강남구도시관리공단,1544-3113,37.464621,127.105538,NaN,2024-07-05,3220000,서울특별시 강남구,NaN,서울특별시 강남구 헌릉로 618길 8
4,122-2-000026,밤고개로21길,공영,노외,NaN,서울특별시 강남구 율현동 529,109,4,미시행,평일+토요일+공휴일,...,강남구도시관리공단,1544-3113,37.473761,127.111886,NaN,2024-07-05,3220000,서울특별시 강남구,NaN,서울특별시 강남구 율현동 529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15130,405-2-000082,노형동 1293-2 공영주차장,공영,노외,NaN,제주특별자치도 제주시 노형동 1293-2,22,1,미시행,평일,...,제주특별자치도 제주시청,NaN,33.490605,126.428533,NaN,2024-07-24,6510000,제주특별자치도 제주시,제주특별자치도 제주시 노형동 1293-2,제주특별자치도 제주시 노형동 1293-2
15131,405-2-000085,노형동 2595-7 공영주차장,공영,노외,NaN,제주특별자치도 제주시 노형동 2595-7,25,1,미시행,평일,...,제주특별자치도 제주시청,NaN,33.491752,126.432236,NaN,2024-07-24,6510000,제주특별자치도 제주시,제주특별자치도 제주시 노형동 2595-7,제주특별자치도 제주시 노형동 2595-7
15132,405-2-000088,칠성상가1 공영주차장,공영,노외,NaN,제주특별자치도 제주시 일도1동 1476-1,19,1,미시행,평일+토요일+공휴일,...,제주특별자치도 제주시청,NaN,33.488693,126.482389,NaN,2024-07-24,6510000,제주특별자치도 제주시,제주특별자치도 제주시 일도1동 1476-1,제주특별자치도 제주시 일도1동 1476-1
15133,405-2-000089,칠성상가2 공영주차장,공영,노외,NaN,제주특별자치도 제주시 일도1동 1401-13,14,1,미시행,평일+토요일+공휴일,...,제주특별자치도 제주시청,NaN,33.508505,126.521667,NaN,2024-07-24,6510000,제주특별자치도 제주시,제주특별자치도 제주시 일도1동 1401-13,제주특별자치도 제주시 일도1동 1401-13


In [49]:
# import re : 특정 규칙을 가진 문자열을 검색하여 치환, 추출 작업을 하는 모듈
import re

def extract_sido(address):
    if pd.isna(address):
        return None
    match = re.match(r"(서울특별시|경기도|인천광역시)", address) # 주소 앞부분에 서울특별시/경기도/인천광역시가 있는지 확인
    return match.group(0) if match else None # match 객체가 있으면, 문자열 전체(주소) 반환

# 반환된 주소값 '시도' 컬럼에 입력
df_cleaned["시도"] = df_cleaned["전체주소"].apply(extract_sido)
df_cleaned.to_csv("확인.csv", index=False, encoding='utf-8-sig')

### 전국주차장 데이터에서 수도권(서울/경기/인천) 데이터만 분리

In [62]:
df_metro = df_cleaned[df_cleaned["시도"].isin(["서울특별시", "경기도", "인천광역시"])]
df_metro.to_csv("수도권_주차장데이터_new.csv", index=False, encoding='utf-8-sig')

### 수도권 주차장 데이터, 공영 주차장 분리 

In [65]:
df_metro = pd.read_csv("수도권_주차장데이터_new.csv", encoding='utf-8-sig')
df_gong = df_metro[df_metro["주차장구분"].isin(["공영"])]
df_gong.to_csv("수도권_공영주차장데이터.csv", index=False, encoding='utf-8-sig')

In [67]:
# 공영주차장 데이터/plotly + mapbox 기반 시각화
# 라이브러리 임포트
import plotly.express as px
import plotly.io as pio
# 데이터 불러오기
df = pd.read_csv("수도권_공영주차장데이터.csv", encoding="utf-8-sig")

pio.renderers.default = 'browser'  # 또는 'browser' / 'iframe' 등 환경에 맞게

# 아래 YOUR_TOKEN_HERE 자리에 mapbox에서 받은 토큰을 입력
px.set_mapbox_access_token("pk.eyJ1IjoicGFtbzIzIiwiYSI6ImNtYWZocTBobzAyZWYya3F4cHRleWo3YjQifQ.gLWenuFkW406_b2vxSrFWw")

# 지도 중심 자동 계산
center_lat = df["위도"].mean()
center_lon = df["경도"].mean()

# 지도 시각화
fig = px.scatter_mapbox(
    df,
    lat="위도",             # 위도 좌표
    lon="경도",             # 경도 좌표
    size="주차구획수",       # 점 크기: 주차 공간 수
    color="시도",           # 색상 구분: 서울/경기/인천
    hover_name="주차장명",   # 마우스오버 시 이름
    hover_data=["주차구획수", "주차장유형", "관리기관명"],
    size_max=20,
    zoom=8,
    title="수도권 공영주차장 위치 및 주차면 수"
)

fig.update_layout(mapbox_style="carto-positron")  # 또는 "open-street-map", "dark", "light"
fig.show()

### 수도권(서울/경기/인천) 충전소 데이터 병합

In [2]:
df_seoul = pd.read_csv("서울_충전소_좌표추가_전체_cleaned.csv", encoding='utf-8-sig')
df_incheon = pd.read_csv("인천광역시_충전소_좌표추가_전체_cleaned.csv", encoding='utf-8-sig')
df_gyeonggi = pd.read_csv("경기도_충전소_좌표추가_전체_cleaned.csv", encoding='utf-8-sig')

# 데이터프레임 병합
merged_df = pd.concat([df_seoul, df_incheon, df_gyeonggi], ignore_index=True)

# 중복 제거 및 인덱스 초기화
merged_df = merged_df.drop_duplicates().reset_index(drop=True)
# 병합 결과 저장 (선택사항)
merged_df.to_csv("수도권_충전소_좌표통합.csv", index=False, encoding="utf-8-sig")

print("✔️ 병합 완료: 수도권_충전소_좌표통합.csv")

### 수도권 충전소 데이터 공영/민영 라벨링 작업

In [38]:
import re  # 정규표현식을 사용하기 위한 모듈

df = pd.read_csv("수도권_충전소_좌표통합_cleaned.csv", encoding='utf-8-sig')

# 공영 키워드 리스트
PUB_KW = [
    # 행정·지자체
    "구청","시청","군청","읍사무소","주민센터","행정복지센터", "국회", "국회의원", "박물관", "민원센터"
    # 공공·공단·공사
    "공영","공원","공원관리","환경공단","시설관리공단","도시공사","관광공사",
    "도로공사","한국도로공사","주차공단","교통공단","공단","공사",
    "한국전력","한전","keplo","전력공사",
    "철도공사","한국철도공사","코레일","국립","국토관리","국가시설","정부청사",
    "소방서","경찰서","우체국","체육센터","도서관",
    # 고속도로·휴게소·공공기관 운영
    "휴게소","고속도로휴게소","만남의광장","고속도로","ex-하이플러스",
    "환경부","국토교통부","한국에너지공단","한국자동차환경협회",
    "도로교통공단","한국교통안전공단","한국환경공단",
    # 공공주차장·공용부지
    "공영주차장","공용주차장","공용","시영","군영"
]

# 민영 키워드 리스트
PRI_KW = [
    # 기존 주거·부동산 키워드
    "아파트","주상복합","오피스텔","아파트형공장","빌라", "빌딩",
    # 수도권 대표 브랜드 아파트
    "래미안","자이","힐스테이트","푸르지오","e편한세상","이편한세상","아이파크",
    "더샵","롯데캐슬","포레나","sk뷰","위브","우미린",
    "센트레빌","한양수자인","반도유보라","경남아너스빌","코아루",
    "데시앙","파크드림","베르디움","아메리칸타운",
    "트리마제","리센츠","디에이치","고덕그라시움","미사강변","센트럴파크",
    # 유통·상업시설
    "백화점","마트","이마트","트레이더스","홈플러스","코스트코","롯데",
    "스타필드","아울렛","쇼핑몰","몰","시장","전통시장",
    # 주유·에너지
    "주유소","충전소","sk","gs","sk에너지","gs칼텍스","s-oil","현대오일뱅크",
    # 커피·F&B
    "카페","스타벅스","투썸","엔제리너스","커피빈","할리스",
    # 의료·숙박
    "병원","의료원","한방병원","치과","의원",
    "호텔","리조트","모텔","게스트하우스",
    # 물류·기업
    "기업","공장","산업단지","물류센터","물류창고","사옥", "현대자동차", "삼성",
    # 주차타워·민영주차장
    "주차타워","민영주차장","사설주차장",
    # 민간 충전 브랜드
    "대영채비","차지비","차지플로","차지링크","even","이브이패스",
    "envico","테슬라","테슬라수퍼차저","현대e-forest","씨티카","에버온",
    "스타코프","소프트베리","피엠그룹","와이비씨","레드스캔","시그넷",
    "이비오","케이웍스","sksig","엔크린","k-lin", "제주전기자동차서비스"
]

# 특수문자 제거 및 소문자화 함수
def clean(text):
    """
    입력된 문자열에서 한글, 숫자, 공백만 남기고
    나머지 문자를 제거한 뒤 소문자로 변환합니다.
    """
    cleaned = re.sub(r'[^가-힣0-9 ]', '', str(text))
    return cleaned.strip().lower()

# 주소와 충전소명 컬럼 전처리
df['addr_clean'] = df['주소'].apply(clean)
df['name_clean'] = df['충전소명'].apply(clean)

In [40]:
# 라벨링 함수 정의
def label_row(row):
    """
    전처리된 주소와 이름을 합쳐 키워드 리스트를 검색하여
    '공영', '민영', '미분류'를 반환합니다.
    """
    text = f"{row['addr_clean']} {row['name_clean']}"
    if any(k in text for k in PUB_KW):
        return '공영'
    if any(k in text for k in PRI_KW):
        return '민영'
    return '미분류'

# owner_type 컬럼 추가
df['owner_type'] = df.apply(label_row, axis=1)

# 결과 확인 (상위 5개 행)
df.head(5)

,충전소명,주소,상세주소,충전기ID,충전방식,충전상태,설치년도,위도,경도,addr_clean,name_clean,owner_type
0,낙성대동주민센터,서울특별시 관악구 낙성대로4가길 5,NaN,1,단독,사용가능,2017,37.510269,127.076398,서울특별시 관악구 낙성대로4가길 5,낙성대동주민센터,공영
1,롯데마트 중계점,서울특별시 노원구 노원로 330,옥상 매장 앞,1,단독,사용가능,2017,37.459645,127.042625,서울특별시 노원구 노원로 330,롯데마트 중계점,민영
2,서울추모공원,서울특별시 서초구 양재대로12길 74,1층 입구,1,단독,사용가능,2017,37.452033,127.042811,서울특별시 서초구 양재대로12길 74,서울추모공원,공영
3,아시아공원 공영주차장,서울특별시 송파구 잠실동 84,우측 끝,1,단독,사용가능,2017,37.510269,127.076398,서울특별시 송파구 잠실동 84,아시아공원 공영주차장,공영
4,아시아공원 공영주차장,서울특별시 송파구 잠실동 84,우측 끝,2,단독,사용가능,2017,37.571988,126.984842,서울특별시 송파구 잠실동 84,아시아공원 공영주차장,공영


In [42]:
df.to_csv("수도권_충전소데이터_민공분리4.csv", index=False, encoding='utf-8-sig')